In [1]:
#Import libaries
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plot
import geopandas as gpd
import plotly.express as px
import json
import plotly.io as pio
import pandas_bokeh
from bokeh.io import output_file, show
from bokeh.layouts import row
from bokeh.plotting import figure
import plotly.graph_objects as go

## Complaint data

In [ ]:
# Load crime complaint into a df

# Import New York cirme complaint data (2006-2021) into a dataframe
df = pd.read_csv('C:/Users/Emili/Desktop/Social data analyse/NYPD_Complaint_Data_Historic.csv')

# Import New York crime complaint data (2022) into a dataframe
df1 = pd.read_csv('C:/Users/Emili/Desktop/Social data analyse/NYPD_Complaint_Data_Current__Year_To_Date_.csv')

# Merge the two df's
df_complaint = pd.concat([df,df1])

In [ ]:
# Create a new dictionary to add a "R" in "MURDER & NON-NEGL. MANSLAUGHTE", so the spelling is the same in the two datasets
murder_dict = {'MURDER & NON-NEGL. MANSLAUGHTE': 'MURDER & NON-NEGL. MANSLAUGHTER'}

# Create a new dictionary for merging "GRAND LARCENY" and "PETIT LARCENY"
larceny_dict = {'GRAND LARCENY': 'LARCENY', 'PETIT LARCENY': 'LARCENY'}

# Using the replace method to correct the spelling of "MURDER & NON-NEGL. MANSLAUGHTER" and merge the "LARCENY" categories and the
df_complaint['OFNS_DESC'] = df_complaint['OFNS_DESC'].replace(murder_dict)
df_complaint['OFNS_DESC'] = df_complaint['OFNS_DESC'].replace(larceny_dict)

# Make a list with the 10 focuscrimes
focuscrimes = set(['SEX CRIMES', 'DANGEROUS DRUGS', 'BURGLARY', 'LARCENY', 'ASSAULT 3 & RELATED OFFENSES', 'MURDER & NON-NEGL. MANSLAUGHTER', 'RAPE', 'ROBBERY', 'PROSTITUTION & RELATED OFFENSES', 'DISORDERLY CONDUCT'])

In [ ]:
# New df_complaint with the 10 focuscrimes only
df_complaint = df_complaint[df_complaint['OFNS_DESC'].isin(focuscrimes)].copy()

# Changing formatting in 'Date' and 'Time' columns + replacing wrong dates with NaT (wrong dates are dates that are outside the valid range of dates and times that pandas can handle fx 1028-06-21)
df_complaint['CMPLNT_FR_DT'] = pd.to_datetime(df_complaint['CMPLNT_FR_DT'], format='%m/%d/%Y', errors='coerce')
df_complaint['CMPLNT_TO_DT'] = pd.to_datetime(df_complaint['CMPLNT_TO_DT'], format='%m/%d/%Y', errors='coerce')

# New dataframe with year 2019-2022 (merge df og df1)
df_complaint = df_complaint.loc[(df_complaint['CMPLNT_FR_DT'] >= '2019-01-01') & (df_complaint['CMPLNT_FR_DT'] < '2023-01-01')].copy()

In [ ]:
# Add "Year" coloumn to the "df_arrest"
df_complaint ['Year'] = df_complaint['CMPLNT_FR_DT'].dt.year

# Add "Month" coloumn to the "df_arrest"
df_complaint ['Month'] = df_complaint['CMPLNT_FR_DT'].dt.month

# Add "Weekday" coloumn to the "df_arrest"
df_complaint ['Weekday'] = df_complaint ['CMPLNT_FR_DT'].dt.weekday

## Arrest data

In [ ]:
# Import New York Arrest data (2006-2021) into a dataframe
df = pd.read_csv('C:/Users/Emili/Desktop/Social data analyse/NYPD_Arrests_Data__Historic_.csv')

# Import New York Arrest data (2022) into a dataframe
df1 = pd.read_csv('C:/Users/Emili/Desktop/Social data analyse/NYPD_Arrest_Data__Year_to_Date_.csv')

# Merge the two df's
df_arrest = pd.concat([df,df1])

# drop the dataframe from the code to free up memory
del df, df1

In [ ]:
# Using the replace method to correct the spelling of "MURDER & NON-NEGL. MANSLAUGHTER" and merge the "LARCENY" categories and the
df_arrest['OFNS_DESC'] = df_arrest['OFNS_DESC'].replace(murder_dict)
df_arrest['OFNS_DESC'] = df_arrest['OFNS_DESC'].replace(larceny_dict)

# New df_arrest with the 10 focuscrimes only
df_arrest = df_arrest[df_arrest['OFNS_DESC'].isin(focuscrimes)].copy()

# Changing formatting in 'Date' and 'Time' columns
df_arrest['ARREST_DATE'] = pd.to_datetime(df_arrest['ARREST_DATE'], format='%m/%d/%Y')

In [ ]:
# New dataframe with year 2019-2022 (merge df og df1)
df_arrest = df_arrest.loc[(df_arrest['ARREST_DATE'] >= '2019-01-01') & (df_arrest['ARREST_DATE'] < '2023-01-01')].copy()

# Add "Year" coloumn to the "df_arrest"
df_arrest ['Year'] = df_arrest['ARREST_DATE'].dt.year

# Add "Month" coloumn to the "df_arrest"
df_arrest ['Month'] = df_arrest['ARREST_DATE'].dt.month

# Add "Weekday" coloumn to the "df_arrest"
df_arrest ['Weekday'] = df_arrest['ARREST_DATE'].dt.weekday

## Borough data errors

Upon examining the boroughs, we discovered rows in the complaint data that contain both null and NaN values. To explore the implications of this, further investigation is warranted.

In [ ]:
#Fill nan's with the value (null) 
df_complaint = df_complaint.assign(BORO_NM=df_complaint['BORO_NM'].fillna("(null)"))

#create a crosstab between Borougs and focuscrimes 
complaint_cross = pd.crosstab(df_complaint.BORO_NM, df_complaint.OFNS_DESC)

#print crosstab
print(complaint_cross)

The NaN and Null values is especially impacting the MURDER & NON-NEGL. MANSLAUGHTER crimecategori. Here is 1209 out of 1778 NaN or Null values. This may have an impact on the distribution of the crimecategori when looking at the boroughs. 

In [ ]:
#New dataframe for murder complaints in Brooklyn 
Comp_murder = df_complaint[df_complaint['OFNS_DESC'] == "MURDER & NON-NEGL. MANSLAUGHTER"].copy()

#Create a crosstab for borougs and year
murder_cross = pd.crosstab(Comp_murder.BORO_NM, Comp_murder.Year)

print(murder_cross)

The crosstabel above show that there is almost no murder complaints that have registret a borough in 2019, 2020 and 2021 suggesting that there might have been some mistakes in the registration process or a procedure of not registrate a borough for murder complaints 

# Choropleth map 

In [ ]:
#Preparing the geometries for Borough Boundaries
Boro = gpd.read_file('C:/Users/Emili/Desktop/Borough Boundaries.geojson')

#The index of the json has to be the borough name
Boro.index = Boro['boro_name']

#Choropleth mapbox accepts a json for the geometries of borough
Boro_json = json.loads(Boro.to_json())
Boro.head()

## Choropleth map complaints per 100,000 residents

In [ ]:
# Group the complaints data by Borough Boundaries and year and count the number of complaints
df_compmap = df_complaint.groupby(['BORO_NM', 'Year']).size().reset_index(name='Number of complaints')

# replace the string '(null)' with NaN values
df_compmap = df_compmap.replace('(null)', pd.NA)

#reset index and dropping NA values 
df_compmap = df_compmap.reset_index().rename(columns={'BORO_NM': 'Borough'}).dropna()

# replace boroughs with capitalized letters so that it matches with the Borough Boundaries
df_compmap['Borough']  = df_compmap['Borough'].str.capitalize().str.replace('island', 'Island')

print(df_compmap)

In [ ]:
# Define a dictionary with the population for each year and borough
population = {'Bronx': {2019: 1418207, 2020: 1461125, 2021: 1421089, 2022: 1379946},
              'Brooklyn': {2019: 2559903, 2020: 2719044, 2021: 2637486, 2022: 2590516},
              'Manhattan': {2019: 1628706, 2020: 1677306, 2021: 1578801, 2022: 1596273},
              'Queens': {2019: 2253858, 2020: 2388586, 2021: 2328141, 2022: 2278029},
              'Staten Island': {2019: 476143, 2020: 494586, 2021: 493484, 2022: 449133}}

#Soruce:
#https://www.nyc.gov/assets/planning/download/pdf/planning-level/nyc-population/population-estimates/current-population-estimates-2022.pdf?r=a
#https://www.nyc.gov/assets/planning/download/pdf/planning-level/nyc-population/population-estimates/current-population-estimates-2019.pdf

# Define a function to calculate complaints for 100,000 residents
def per_capita_complaints(row):
    borough = row['Borough']
    year = row['Year']
    complaints = row['Number of complaints']
    
    # Calculate number of complaints per 100,000 residents
    return complaints / population[borough][year] * 100000

# Use the function to calculate complaints per 100,000 residents for each row
df_compmap['Complaints per 100k'] = df_compmap.apply(per_capita_complaints, axis=1).apply(round).apply(int)

print(df_compmap)

In [ ]:
#using plotly for an animated choropleth map
fig = px.choropleth_mapbox(data_frame=df_compmap,
                           geojson=Boro_json,
                           locations=df_compmap.Borough,
                           color='Complaints per 100k',
                           center={"lat": 40.7250, "lon": -73.9851},
                           mapbox_style='carto-positron',
                           zoom=9,
                           color_continuous_scale='blues',
                           range_color=(1000, 5500),
                           animation_frame='Year',
                           width=800,
                           height=600)

# Set the animation frame to 2019
fig.layout.sliders[0].active = 8


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Display the plot
fig.show()


In [ ]:
# Write to HTML file with animation initially paused
pio.write_html(fig, file='Complaints_map_per_100.html', auto_open=True, auto_play=False)

## Choropleth map arrest per 100,000 residents

In [ ]:
# Group the arrests data by borough and year and count the number of arrests
df_arrmap = df_arrest.groupby(['ARREST_BORO', 'Year']).size().reset_index(name='Number of arrests')

df_arrmap = df_arrmap.reset_index().rename(columns={'ARREST_BORO': 'ARREST_BORO'})

# Replacing ARREST_BORO names with borough names 
df_arrmap["ARREST_BORO"].replace({"B": "Bronx", "S": "Staten Island","M": "Manhattan"
                               ,"K": "Brooklyn","Q": "Queens"}, inplace=True)

print(df_arrmap)

In [ ]:
# Define a dictionary with the population for each year and borough
population = {'Bronx': {2019: 1418207, 2020: 1461125, 2021: 1421089, 2022: 1379946},
              'Brooklyn': {2019: 2559903, 2020: 2719044, 2021: 2637486, 2022: 2590516},
              'Manhattan': {2019: 1628706, 2020: 1677306, 2021: 1578801, 2022: 1596273},
              'Queens': {2019: 2253858, 2020: 2388586, 2021: 2328141, 2022: 2278029},
              'Staten Island': {2019: 476143, 2020: 494586, 2021: 493484, 2022: 449133}}

#  Define a function to calculate arrest for 100,000 residents
def per_capita_arrests(row):
    borough = row['ARREST_BORO']
    year = row['Year']
    arrests = row['Number of arrests']
    
    # calculate arrests per 100.000 residents
    return arrests / population[borough][year] * 100000

# Use the function to calculate arrests per 100,000 residents for each row
df_arrmap['Arrests per 100k'] = df_arrmap.apply(per_capita_arrests, axis=1).apply(round).apply(int)

print(df_arrmap)

In [ ]:
#using plotly for an animated choropleth map
fig2 = px.choropleth_mapbox(data_frame=df_arrmap,
                           geojson=Boro_json,
                           locations=df_arrmap.ARREST_BORO,
                           color='Arrests per 100k',
                           center={"lat": 40.7250, "lon": -73.9851},
                           mapbox_style='carto-positron',
                           zoom=9,
                           color_continuous_scale='reds',
                           range_color=(75, 2500),
                           animation_frame='Year',
                           width=800,
                           height=600)

# Set the animation frame to 2019
fig2.layout.sliders[0].active = 8


fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Display the plot
fig2.show()

In [ ]:
#Write to html file
pio.write_html(fig2, file='Arrest_map_per_100.html', auto_open=True, auto_play=False)

# Boroughs and focuscrimes - complaint

In [ ]:
#New complaint dataframe for each borough
Comp_Bronx2= df_complaint[df_complaint['BORO_NM'] == "BRONX"].copy()
Comp_Brooklyn2= df_complaint[df_complaint['BORO_NM'] == "BROOKLYN"].copy()
Comp_Manhattan2= df_complaint[df_complaint['BORO_NM'] == "MANHATTAN"].copy()
Comp_Queens2= df_complaint[df_complaint['BORO_NM'] == "QUEENS"].copy()
Comp_Staten2 = df_complaint[df_complaint['BORO_NM'] == "STATEN ISLAND"].copy()

#New pivot for each borough
df_bronx_pivot = pd.pivot_table(Comp_Bronx2, values='CMPLNT_NUM', index='Year', columns='OFNS_DESC', aggfunc='count')
df_brooklyn_pivot = pd.pivot_table(Comp_Brooklyn2, values='CMPLNT_NUM', index='Year', columns='OFNS_DESC', aggfunc='count')
df_manhattan_pivot = pd.pivot_table(Comp_Manhattan2, values='CMPLNT_NUM', index='Year', columns='OFNS_DESC', aggfunc='count')
df_queens_pivot = pd.pivot_table(Comp_Queens2, values='CMPLNT_NUM', index='Year', columns='OFNS_DESC', aggfunc='count')
df_staten_pivot = pd.pivot_table(Comp_Staten2, values='CMPLNT_NUM', index='Year', columns='OFNS_DESC', aggfunc='count')

In [ ]:
fig = go.Figure()   

# Adding data traces for each focus crime
for i, focuscrime in enumerate(df_bronx_pivot.columns):
    visible = [False] * (5 * len(df_bronx_pivot.columns))
    if i == 0:
        visible[0] = False
        visible[1] = True
        visible[2] = True
        visible[3] = True
        visible[4] = True
        
    fig.add_trace(
        go.Bar(
            x=df_bronx_pivot.index,
            y=df_bronx_pivot[focuscrime],
            name=f"Bronx: {focuscrime} ",
            hovertemplate='%{y}',
            marker=dict(color='#0047ab'),
            visible=visible[0]
        )
    )
   
    fig.add_trace(
        go.Bar(
            x=df_brooklyn_pivot.index,
            y=df_brooklyn_pivot[focuscrime],
            name=f"Brooklyn: {focuscrime}",
            marker=dict(color='#91c4e6'),
            hovertemplate='%{y}',
            visible=visible[1]
        )
    )
    fig.add_trace(
        go.Bar(
            x=df_manhattan_pivot.index,
            y=df_manhattan_pivot[focuscrime],
            name=f"Manhattan: {focuscrime}",
            marker=dict(color='#f8b17d'),
            hovertemplate='%{y}',
            visible=visible[2]
        )
    ) 
    fig.add_trace(
        go.Bar(
            x=df_queens_pivot.index,
            y=df_queens_pivot[focuscrime],
            name=f"Queens: {focuscrime}",
            marker=dict(color='#ffd571'),
            hovertemplate='%{y}',
            visible=visible[3]
        )
     )
    fig.add_trace(
        go.Bar(
            x=df_staten_pivot.index,
            y=df_staten_pivot[focuscrime],
            name=f"Staten Island: {focuscrime}",
            marker=dict(color='#a3d9b1'),
            hovertemplate='%{y}',
            visible=visible[4]
        )
     )
    
# Make all data visible by default
fig.update_traces(visible=True)

# Create dropdown menu
dropdown_menu = []
for focuscrime in ['Select a focuscrime'] + list(df_bronx_pivot.columns):
    visible = [False] * (5 * len(df_bronx_pivot.columns))
    if focuscrime == 'Select a focuscrime':
        title_text = "<b>Yearly number of complaints in the five boroughs<b>"
    else:
        idx = df_bronx_pivot.columns.get_loc(focuscrime)
        visible[5 * idx] = True
        visible[5 * idx + 1] = True
        visible[5 * idx + 2] = True
        visible[5 * idx + 3] = True
        visible[5 * idx + 4] = True
        title_text = f"<b>Yearly number of complaints in the five boroughs"
    dropdown_menu.append(
        dict(
            label=focuscrime,
            method='update',
            args=[
                {'visible': visible},
                {'title': title_text}
            ]
        )
    )

# Add dropdown menu to layout
fig.update_layout(
    title={
        'text': "<b>Yearly number of complaints in the five boroughs<b>",
        'x': 0.65,  # Centered title
        'xanchor': 'center',
    },
    updatemenus=[dict(
        buttons=dropdown_menu,
        direction='down',
        pad={'r': 10, 't': 10},
        showactive=True,
        x=0.0,
        xanchor='left',
        y=1.2,
        yanchor='top'
    )],
    #title='<b>Yearly number of complaints in the five boroughs, 2019-2022<b>',
    #title_x=0.56,
    legend=dict(title="<b>Click to select/deselect:<b>", font=dict(size=11)),
    margin=dict(t=50),
    xaxis=dict(tickmode='linear', tick0=2019, dtick=1),
    yaxis=dict(title='No. of complaints', ticksuffix=' ', title_font=dict(size=14, color='#444')),
    hovermode='x unified'  # To show all hover data at once
)


fig.show()

In [ ]:
#Write to html file
pio.write_html(fig, file='Complaint_Borough.html', auto_open=True)

# Boroughs and focuscrimes - complaint

In [ ]:
#New arrest dataframe for each borough
Arrest_Bronx= df_arrest[df_arrest['ARREST_BORO'] == "B"].copy()
Arrest_Brooklyn= df_arrest[df_arrest['ARREST_BORO'] == "K"].copy()
Arrest_Manhattan= df_arrest[df_arrest['ARREST_BORO'] == "M"].copy()
Arrest_Queens= df_arrest[df_arrest['ARREST_BORO'] == "Q"].copy()
Arrest_Staten = df_arrest[df_arrest['ARREST_BORO'] == "S"].copy()

#New pivot for each borough
arrest_bronx_pivot = pd.pivot_table(Arrest_Bronx, values='ARREST_KEY', index='Year', columns='OFNS_DESC', aggfunc='count')
arrest_brooklyn_pivot = pd.pivot_table(Arrest_Brooklyn, values='ARREST_KEY', index='Year', columns='OFNS_DESC', aggfunc='count')
arrest_manhattan_pivot = pd.pivot_table(Arrest_Manhattan, values='ARREST_KEY', index='Year', columns='OFNS_DESC', aggfunc='count')
arrest_queens_pivot = pd.pivot_table(Arrest_Queens, values='ARREST_KEY', index='Year', columns='OFNS_DESC', aggfunc='count')
arrest_staten_pivot = pd.pivot_table(Arrest_Staten, values='ARREST_KEY', index='Year', columns='OFNS_DESC', aggfunc='count')

In [ ]:
fig = go.Figure()   

# Adding data traces for each focus crime
for i, focuscrime in enumerate(arrest_bronx_pivot.columns):
    visible = [False] * (5 * len(arrest_bronx_pivot.columns))
    if i == 0:
        visible[0] = False
        visible[1] = True
        visible[2] = True
        visible[3] = True
        visible[4] = True
        
    fig.add_trace(
        go.Bar(
            x=arrest_bronx_pivot.index,
            y=arrest_bronx_pivot[focuscrime],
            name=f"Bronx: {focuscrime} ",
            hovertemplate='%{y}',
            marker=dict(color='#0047ab'),
            visible=visible[0]
        )
    )
   
    fig.add_trace(
        go.Bar(
            x=arrest_brooklyn_pivot.index,
            y=arrest_brooklyn_pivot[focuscrime],
            name=f"Brooklyn: {focuscrime}",
            marker=dict(color='#91c4e6'),
            hovertemplate='%{y}',
            visible=visible[1]
        )
    )
    fig.add_trace(
        go.Bar(
            x=arrest_manhattan_pivot.index,
            y=arrest_manhattan_pivot[focuscrime],
            name=f"Manhattan: {focuscrime}",
            marker=dict(color='#f8b17d'),
            hovertemplate='%{y}',
            visible=visible[2]
        )
    ) 
    fig.add_trace(
        go.Bar(
            x=arrest_queens_pivot.index,
            y=arrest_queens_pivot[focuscrime],
            name=f"Queens: {focuscrime}",
            marker=dict(color='#ffd571'),
            hovertemplate='%{y}',
            visible=visible[3]
        )
     )
    fig.add_trace(
        go.Bar(
            x=arrest_staten_pivot.index,
            y=arrest_staten_pivot[focuscrime],
            name=f"Staten Island: {focuscrime}",
            marker=dict(color='#a3d9b1'),
            hovertemplate='%{y}',
            visible=visible[4]
        )
     )
    
# Make all data visible by default
fig.update_traces(visible=True)

# Create dropdown menu
dropdown_menu = []
for focuscrime in ['Select a focuscrime'] + list(arrest_bronx_pivot.columns):
    visible = [False] * (5 * len(arrest_bronx_pivot.columns))
    if focuscrime == 'Select a focuscrime':
        title_text = "<b>Yearly number of arrests in the five boroughs<b>"
    else:
        idx = arrest_bronx_pivot.columns.get_loc(focuscrime)
        visible[5 * idx] = True
        visible[5 * idx + 1] = True
        visible[5 * idx + 2] = True
        visible[5 * idx + 3] = True
        visible[5 * idx + 4] = True
        title_text = f"<b>Yearly number of arrests in the five boroughs"
    dropdown_menu.append(
        dict(
            label=focuscrime,
            method='update',
            args=[
                {'visible': visible},
                {'title': title_text}
            ]
        )
    )

# Add dropdown menu to layout
fig.update_layout(
    title={
        'text': "<b>Yearly number of arrests in the five boroughs<b>",
        'x': 0.65,  # Centered title
        'xanchor': 'center',
    },
    updatemenus=[dict(
        buttons=dropdown_menu,
        direction='down',
        pad={'r': 10, 't': 10},
        showactive=True,
        x=0.0,
        xanchor='left',
        y=1.2,
        yanchor='top'
    )],
    #title='<b>Yearly number of arrests in the five boroughs, 2019-2022<b>',
    #title_x=0.56,
    legend=dict(title="<b>Click to select/deselect:<b>", font=dict(size=11)),
    margin=dict(t=50),
    xaxis=dict(tickmode='linear', tick0=2019, dtick=1),
    yaxis=dict(title='No. of arrests', ticksuffix=' ', title_font=dict(size=14, color='#444')),
    hovermode='x unified'  # To show all hover data at once
)


fig.show()

In [ ]:
#Write to html file
pio.write_html(fig, file='Arrest_borough.html', auto_open=True)